In [4]:
import datetime
import os
import numpy as np
from data_process import get_data, write_data
from image_utils import plot_show
from data_process import OneHot
from model import Generator
import tensorflow as tf
import pandas as pd

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # 使用CPU

all_start = datetime.datetime.now()  # 程序开始运行

# 超参数设定
sample_size = 96  # 一次性学习的样本大小
hidden = 100  # LSTM层的神经元个数
batch_size = 1  # LSTM层的batch_size
time_step = 96  # LSTM层的time_step
learn_set = 1000  # 迭代次数列表
cond_dim = 2  # 条件值
latent_dim = 1  # latent space 维度
num_run = 1  # 运行的次数
num_gen_once = 1  # 单次生成序列
LR = 0.001  # 学习率
# 数据处理部分
data = './data'  # 读取data文件夹内的数据集，数据集是一个月内的停车数据 每十五分钟取一个点，每天96个点
sample_set, index, num_seq = get_data(data, sample_size)
index_list = [i for i in range(num_seq)]
# g_data = train(sample_set, index, sample_size, learn_set, batch_size, hidden, time_step, num_seq,

In [3]:
from NoiseDistribution import NoiseDistribution
noiseDistri = NoiseDistribution(2) # 2

In [4]:
lab_to_one = OneHot(num_seq, sample_size, cond_dim) # 30, 96, 2
index_ = lab_to_one.one_hot_index(index[0])

In [5]:
inputs = np.reshape(sample_set[0][0], newshape=[batch_size, sample_size, 1])  # 1*96*1
conditions = np.reshape(index_[0], newshape=[batch_size, sample_size, cond_dim])  # 1*96*2
fake_inputs = noiseDistri.sample(sample_size)

In [54]:
z = tf.concat([fake_inputs, conditions], axis=2)  # 1*96*3
z = tf.reshape(z, shape=[-1, z.get_shape()[2]])  # 96*3
z = generator.dense_z(z)  # 96*10
y = generator.dense_y(z)  # 96*100
y = tf.expand_dims(y, axis=0)  # 1*96*100

In [57]:
generator.lstm = tf.keras.layers.LSTM(100, activation='relu', return_sequences=True)
x = generator.lstm(y)  # 1*96*100
x = tf.squeeze(x, axis=0)  # 96*100
x = generator.final_dense(x) # 96*1

In [10]:
import pandas as pd

def get_data_(data_path, batch_size=96):
    """
    Read data
    """
    total_batch = 0
    file_names = os.listdir(data_path)
    day_parking_rate = []
    one_hot = OneHot(batch_size)  # 30, 96, 2
    for file_name in file_names:
        if os.path.isdir(file_name):
            continue
        df = pd.read_excel(data_path + "/" + file_name)
        total_batch += int(len(df) / batch_size)
        # 96 points for each day
        day_parking_rate.append(
            list(map(lambda x: one_hot.encode(int(x[1])),
                     df.values[day * batch_size:day * batch_size + batch_size]))
            for day in range(total_batch))
    return day_parking_rate, total_batch

In [6]:
batch_size = 96
data_path = './data'
total_batch = 0
file_names = os.listdir(data_path)
day_parking_rate = []
one_hot = OneHot(batch_size)  # 30, 96, 2
for file_name in file_names:
    if os.path.isdir(file_name):
        continue
    df = pd.read_excel(data_path + "/" + file_name)
    total_batch += int(len(df) / batch_size)
    # 96 points for each day
#     for day in range(total_batch):
#         day_points = df.values[day * batch_size:day * batch_size + batch_size]
#         encoded_day_points = list(map(lambda x: [x[0], one_hot.encode(int(x[1]))] ,day_points))
#         print(encoded_day_points)
#         day_parking_rate.append(encoded_day_points)
    day_parking_rate.append(
        [list(map(lambda x: [x[0]] + one_hot.encode(int(x[1])),
                 df.values[day * batch_size:day * batch_size + batch_size]))
        for day in range(total_batch)])

In [17]:
generator = Generator()
generator(np.array(day_parking_rate[0][0]))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



<tf.Tensor: shape=(1, 96, 1), dtype=float32, numpy=
array([[[0.00266647],
        [0.00588322],
        [0.00925644],
        [0.01224977],
        [0.01489185],
        [0.01718767],
        [0.01934767],
        [0.02134032],
        [0.02298829],
        [0.02446583],
        [0.02575091],
        [0.026807  ],
        [0.02767884],
        [0.02840183],
        [0.02907784],
        [0.02969771],
        [0.03013641],
        [0.03048624],
        [0.03077262],
        [0.03097742],
        [0.03116101],
        [0.03125547],
        [0.03128665],
        [0.03130845],
        [0.03132291],
        [0.03129552],
        [0.03127518],
        [0.03105693],
        [0.03098945],
        [0.03087263],
        [0.03079901],
        [0.03054139],
        [0.02999026],
        [0.0295592 ],
        [0.02892887],
        [0.02809872],
        [0.02707573],
        [0.02617699],
        [0.02532835],
        [0.02448333],
        [0.02377984],
        [0.02319602],
        [0.02271902],
  

In [33]:
day_parking_rate[0][0]

[[0.28879310344827586, 1.0, 0.0],
 [0.2629310344827586, 1.0, 0.0],
 [0.25, 1.0, 0.0],
 [0.25, 1.0, 0.0],
 [0.24568965517241378, 1.0, 0.0],
 [0.2413793103448276, 1.0, 0.0],
 [0.22413793103448276, 1.0, 0.0],
 [0.20689655172413793, 1.0, 0.0],
 [0.2025862068965517, 1.0, 0.0],
 [0.1896551724137931, 1.0, 0.0],
 [0.1810344827586207, 1.0, 0.0],
 [0.17672413793103448, 1.0, 0.0],
 [0.1724137931034483, 1.0, 0.0],
 [0.16810344827586207, 1.0, 0.0],
 [0.15948275862068967, 1.0, 0.0],
 [0.15086206896551724, 1.0, 0.0],
 [0.15086206896551724, 1.0, 0.0],
 [0.14655172413793102, 1.0, 0.0],
 [0.14224137931034483, 1.0, 0.0],
 [0.14224137931034483, 1.0, 0.0],
 [0.13793103448275862, 1.0, 0.0],
 [0.14224137931034483, 1.0, 0.0],
 [0.14655172413793102, 1.0, 0.0],
 [0.14655172413793102, 1.0, 0.0],
 [0.14655172413793102, 1.0, 0.0],
 [0.15086206896551724, 1.0, 0.0],
 [0.15086206896551724, 1.0, 0.0],
 [0.16379310344827586, 1.0, 0.0],
 [0.15948275862068967, 1.0, 0.0],
 [0.16379310344827586, 1.0, 0.0],
 [0.163793103448

In [22]:
from NoiseDistribution import NoiseDistribution
noise = NoiseDistribution(2)
noise.sample(96)[0]

array([-1.98160114])